In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from iqoptionapi.stable_api import IQ_Option
import logging
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import preprocessing
from sklearn import utils

# Fazendo login no IQ Option
email = str(input())
senha = str(input())
iq = IQ_Option(email, senha)
iq.connect()

if iq.check_connect() is True:
    print("Logado com sucesso!")

end_from_time = time.time()
listafinal = []

Logado com sucesso!


In [ ]:
# Obtendo os dados de velas (candles)
for i in range(500):
    candles = iq.get_candles(ACTIVES='EURUSD', interval=900, count=1000, endtime=end_from_time)
    listafinal = listafinal + candles
    end_from_time = int(candles[0]["from"]) - 1

In [6]:
import openpyxl
# Criando DataFrame
listafinal = pd.DataFrame(listafinal)
listafinal = listafinal.drop(columns=['id', 'at', 'to'])

In [7]:

# Convertendo os dados para float
listafinal = listafinal.astype(float)
listafinal = listafinal.sort_values('from')
print(listafinal)
#Salva o dataset em Excel para backup
listafinal.to_excel('dataset.xlsx')


                from      open     close       min       max  volume
102000  1.587640e+09  1.077545  1.079190  1.077370  1.079190  2791.0
102001  1.587640e+09  1.079195  1.078145  1.077785  1.079235  2645.0
102002  1.587641e+09  1.078135  1.077935  1.077510  1.078465  2590.0
102003  1.587642e+09  1.077920  1.076560  1.076365  1.077980  2407.0
102004  1.587643e+09  1.076570  1.077175  1.076540  1.077560  2678.0
...              ...       ...       ...       ...       ...     ...
995     1.717185e+09  1.084655  1.085130  1.084600  1.085135  2431.0
996     1.717186e+09  1.085120  1.084995  1.084770  1.085215  1860.0
997     1.717186e+09  1.084995  1.084740  1.084655  1.085050  1614.0
998     1.717187e+09  1.084750  1.084940  1.084615  1.084970  1689.0
999     1.717188e+09  1.084945  1.084825  1.084790  1.085100  1800.0

[102544 rows x 6 columns]


In [8]:
listafinal = pd.read_excel('dataset.xlsx')
listafinal = listafinal.drop(columns=['Unnamed: 0'])
print(listafinal)

              from      open     close       min       max  volume
0       1587639600  1.077545  1.079190  1.077370  1.079190    2791
1       1587640500  1.079195  1.078145  1.077785  1.079235    2645
2       1587641400  1.078135  1.077935  1.077510  1.078465    2590
3       1587642300  1.077920  1.076560  1.076365  1.077980    2407
4       1587643200  1.076570  1.077175  1.076540  1.077560    2678
...            ...       ...       ...       ...       ...     ...
102539  1717184700  1.084655  1.085130  1.084600  1.085135    2431
102540  1717185600  1.085120  1.084995  1.084770  1.085215    1860
102541  1717186500  1.084995  1.084740  1.084655  1.085050    1614
102542  1717187400  1.084750  1.084940  1.084615  1.084970    1689
102543  1717188300  1.084945  1.084825  1.084790  1.085100    1800

[102544 rows x 6 columns]


In [9]:
#Cria coluna de valor de fechamento da vela seguinte...
listafinal['closeNextCandle'] = 0
print(listafinal)

              from      open     close       min       max  volume  \
0       1587639600  1.077545  1.079190  1.077370  1.079190    2791   
1       1587640500  1.079195  1.078145  1.077785  1.079235    2645   
2       1587641400  1.078135  1.077935  1.077510  1.078465    2590   
3       1587642300  1.077920  1.076560  1.076365  1.077980    2407   
4       1587643200  1.076570  1.077175  1.076540  1.077560    2678   
...            ...       ...       ...       ...       ...     ...   
102539  1717184700  1.084655  1.085130  1.084600  1.085135    2431   
102540  1717185600  1.085120  1.084995  1.084770  1.085215    1860   
102541  1717186500  1.084995  1.084740  1.084655  1.085050    1614   
102542  1717187400  1.084750  1.084940  1.084615  1.084970    1689   
102543  1717188300  1.084945  1.084825  1.084790  1.085100    1800   

        closeNextCandle  
0                     0  
1                     0  
2                     0  
3                     0  
4                     0  
...

In [15]:
#Inverte a lista (executar só 1 vez)
#Depois passa por todas linhas e salva o valor de fechamento, para adicionar ao closeNextCandle da linha seguinte...
listafinal = listafinal.iloc[::-1]
for index, row in listafinal.iterrows():
    if index == 102543:
        listafinal.at[index, 'closeNextCandle'] = 0
    else:
        listafinal.at[index, 'closeNextCandle'] = valor
    valor = row['close']
print(listafinal)
        
    

C:\Users\Meu Computador\AppData\Local\Temp\ipykernel_15892\3736081789.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.084825' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  listafinal.at[index, 'closeNextCandle'] = valor


              from      open     close       min       max  volume  \
102543  1717188300  1.084945  1.084825  1.084790  1.085100    1800   
102542  1717187400  1.084750  1.084940  1.084615  1.084970    1689   
102541  1717186500  1.084995  1.084740  1.084655  1.085050    1614   
102540  1717185600  1.085120  1.084995  1.084770  1.085215    1860   
102539  1717184700  1.084655  1.085130  1.084600  1.085135    2431   
...            ...       ...       ...       ...       ...     ...   
4       1587643200  1.076570  1.077175  1.076540  1.077560    2678   
3       1587642300  1.077920  1.076560  1.076365  1.077980    2407   
2       1587641400  1.078135  1.077935  1.077510  1.078465    2590   
1       1587640500  1.079195  1.078145  1.077785  1.079235    2645   
0       1587639600  1.077545  1.079190  1.077370  1.079190    2791   

        closeNextCandle  
102543         0.000000  
102542         1.084825  
102541         1.084940  
102540         1.084740  
102539         1.084995  
...

In [16]:
#Transforma os valores em binário para regressão logística
for index, row in listafinal.iterrows():
    if row["closeNextCandle"] > row["close"]:
        listafinal.at[index, 'closeNextCandle'] = 1
    else:
        listafinal.at[index, 'closeNextCandle'] = 0
print(listafinal)


              from      open     close       min       max  volume  \
102543  1717188300  1.084945  1.084825  1.084790  1.085100    1800   
102542  1717187400  1.084750  1.084940  1.084615  1.084970    1689   
102541  1717186500  1.084995  1.084740  1.084655  1.085050    1614   
102540  1717185600  1.085120  1.084995  1.084770  1.085215    1860   
102539  1717184700  1.084655  1.085130  1.084600  1.085135    2431   
...            ...       ...       ...       ...       ...     ...   
4       1587643200  1.076570  1.077175  1.076540  1.077560    2678   
3       1587642300  1.077920  1.076560  1.076365  1.077980    2407   
2       1587641400  1.078135  1.077935  1.077510  1.078465    2590   
1       1587640500  1.079195  1.078145  1.077785  1.079235    2645   
0       1587639600  1.077545  1.079190  1.077370  1.079190    2791   

        closeNextCandle  
102543              0.0  
102542              0.0  
102541              1.0  
102540              0.0  
102539              0.0  
...

In [17]:
listafinal.to_excel('datasetready.xlsx')

In [19]:
# Carregar o dataset
df = pd.read_excel('datasetready.xlsx')
df = df.drop(columns=['Unnamed: 0'])
# Exibir as primeiras linhas do dataframe para verificar a estrutura
df.head()

,from,open,close,min,max,volume,closeNextCandle
0,1717188300,1.084945,1.084825,1.084790,1.085100,1800,0
1,1717187400,1.084750,1.084940,1.084615,1.084970,1689,0
2,1717186500,1.084995,1.084740,1.084655,1.085050,1614,1
3,1717185600,1.085120,1.084995,1.084770,1.085215,1860,0
4,1717184700,1.084655,1.085130,1.084600,1.085135,2431,0


In [26]:
from sklearn.preprocessing import StandardScaler

# Selecionar as features e o target
features = ['open', 'max', 'min', 'close', 'volume']
target = 'closeNextCandle'

# Normalizar as features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])

# Normalizar o target
target_scaler = StandardScaler()
y_scaled = target_scaler.fit_transform(df[[target]])

# Criar sequências de tempo
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10
X_seq, y_seq = create_sequences(X_scaled, y_scaled, time_steps)

# Dividir em treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Construir o modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(time_steps, X_train.shape[2])))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100


c:\Users\Meu Computador\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2051/2051 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - loss: 1.0044 - val_loss: 1.0002
Epoch 2/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.0005 - val_loss: 0.9999
Epoch 3/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 1.0001 - val_loss: 0.9998
Epoch 4/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - loss: 0.8465 - val_loss: 0.6006
Epoch 5/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.5570 - val_loss: 0.5562
Epoch 6/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.5216 - val_loss: 0.4995
Epoch 7/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.5001 - val_loss: 0.4742
Epoch 8/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.4693 - val_loss: 0.4576
Epoch 9/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.4409 - val_loss: 0.4166
Epoch 10/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.4039 - val_loss: 0.3797
Epoch 11/100
2051/2051 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step - loss: 0.3700 - val_loss: 0.3490
Epoch 12/100
2051/20

In [32]:
# Avaliar o modelo
loss = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')

641/641 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0876
Loss: 0.08555592596530914


In [ ]:
import pickle

